In [ ]:
## Step 1: Install and Import Packages


In [4]:
!pip install requests beautifulsoup4 pandas 
!pip install selenium webdriver-manager



In [48]:
# 🔧 System + File Handling
import os
import time
import re
import sys
import difflib

# 🗓️ Date Handling
from datetime import datetime, timedelta

# 🧠 Data Handling
import pandas as pd

# 🕷️ HTTP & HTML Parsing
import requests
from bs4 import BeautifulSoup

# 🤖 Fuzzy String Matching
from fuzzywuzzy import process


In [49]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager

options = Options()
options.add_argument("--headless")  # Optional: comment out if you want to see browser
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

# This will automatically download the right chromedriver version
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)


In [50]:
import sys
print(sys.executable)


/Library/Frameworks/Python.framework/Versions/3.12/bin/python3


In [51]:
BASE_URL = "https://www.thesmilingseahorse.com"
BLOG_URL = BASE_URL + "/blog"


In [52]:
keywords = [
    "manta", "devil ray", "mobula", "eagle ray", "stingray", "ray", 
    "shark", "reef shark", "blacktip", "whitetip", "leopard shark", 
    "nurse shark", "zebra shark", "guitar shark", "bull shark", 
    "hammerhead", "oceanic whitetip", "tiger shark", "whale shark", "sharks", "rays", "silvertip", "requiem", 
    "oceanic manta", "mantas", "sharks", "guitarfish", "guitarshark"
]


In [53]:
dive_sites = [
    # Thailand: Similan, Surin, Koh Bon/Tachai
    "Richelieu Rock", "Koh Bon", "Koh Tachai", "Similan Islands", "Surin Islands",
    "Elephant Head Rock", "East of Eden", "Three Trees", "North Point",
    "West of Eden", "Christmas Point", "Snapper Alley", "Breakfast Bend",
    "Tuna Wreck", "Anita's Reef", "Beacon Reef", "Shark Fin Reef",
    
    # Thailand South: Hin Daeng / Hin Muang
    "Hin Daeng", "Hin Muang", "Koh Haa", "Koh Bida Nok", "Koh Bida Nai", "Shark Point",
    "King Cruiser Wreck", "Anemone Reef", "Phi Phi Islands",

    # Myanmar: Mergui Archipelago & Burma Banks
    "Black Rock", "North Twin", "South Twin", "Square Rock", "Shark Cave", "Square",
    "Seafan Forest", "Eagle Rock", "Western Rocky", "Volcano", "Burma Banks",
    "Crayfish Cave", "Split Rock", "Diver’s Rock", "The Arch", "The Pinnacle",
    "Secret Reef", "The Tower", "Cavern Island", "Rocky Island", "Submarine",
    "Roe Bank", "Row Bank", "Silvertip Bank", "Rainbow Reef", "Shark Reef", "Coral Bank", "Big Bank",      
    "North Bank", "Western Bank", "South Bank", "Central Bank", "Staghorn Bank"
]


In [54]:
!pip3 install 'fuzzywuzzy[speedup]'

In [55]:
from fuzzywuzzy import process

def match_dive_site_fuzzy(text_block, site_list, threshold=0.7):
    text_lower = text_block.lower()
    best_match = ""
    best_score = 0

    for site in site_list:
        site_lower = site.lower()
        if site_lower in text_lower:
            return site  # exact-ish match
        else:
            ratio = difflib.SequenceMatcher(None, site_lower, text_lower).ratio()
            if ratio > best_score and ratio >= threshold:
                best_match = site
                best_score = ratio

    return best_match



In [56]:
def extract_trip_start_date(text):
    import re
    from datetime import datetime

    # Remove ordinal suffixes (23rd -> 23)
    text = re.sub(r'(\d{1,2})(st|nd|rd|th)', r'\1', text)

    # Try several date patterns
    patterns = [
        r'(\d{1,2})\s*(?:–|-|to)\s*(\d{1,2})\s+([A-Z][a-z]+)\s+(\d{4})',
        r'([A-Z][a-z]+)\s+(\d{1,2})\s*(?:–|-|to)\s*(\d{1,2}),?\s+(\d{4})',
        r'([A-Z][a-z]+)\s+(\d{1,2}),?\s+(\d{4})'
    ]
    for pattern in patterns:
        match = re.search(pattern, text)
        if match:
            try:
                if len(match.groups()) == 4:
                    if pattern.startswith('('):
                        day, _, month, year = match.groups()
                    else:
                        month, day, _, year = match.groups()
                elif len(match.groups()) == 3:
                    month, day, year = match.groups()
                return datetime.strptime(f"{month} {day}, {year}", "%B %d, %Y").date()
            except:
                continue
    return None

In [57]:
def split_by_day_sections(text):
    import re
    pattern = r'(Day\s+(\d+)[^\n]*)\n'
    matches = list(re.finditer(pattern, text, flags=re.IGNORECASE))
    sections = []
    for i in range(len(matches)):
        start = matches[i].end()
        end = matches[i+1].start() if i+1 < len(matches) else len(text)
        day_header = matches[i].group(1).strip()
        day_number = int(matches[i].group(2))
        content = text[start:end].strip()
        sections.append((day_number, day_header, content))
    return sections


In [11]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time

def get_blog_links():
    base_url = "https://www.thesmilingseahorse.com"
    links = set()
    page = 1

    driver = webdriver.Safari()

    while True:
        url = f"{base_url}/blog" if page == 1 else f"{base_url}/blog/previous/{page}"
        print(f"\nScraping index page: {url}")
        driver.get(url)
        time.sleep(3)

        # 👋 Remove popup
        try:
            driver.execute_script("""
                let popup = document.getElementById('PopupSignupForm_0');
                if (popup) popup.remove();
            """)
            print("Popup forcibly removed ✅")
        except:
            print("No popup removed or not found.")

        # ✅ Wait for blog content to load
        try:
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, "blog-title-link"))
            )
            print("✅ Blog post links loaded!")
        except:
            print("❌ Blog post links not found — stopping.")
            break

        # 🔽 Scroll to trigger lazy content
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3)

        # 🌐 Parse blog post links from main content only
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        main_content = soup.select('div#wsite-content')
        post_links = main_content[0].select('h2.blog-title a.blog-title-link') if main_content else []

        if not post_links:
            print(f"No blog post titles in body on page {page}. Stopping.")
            break

        for a in post_links:
            href = a['href']
            # Normalize relative and protocol-relative links
            if href.startswith('//'):
                href = 'https:' + href
            elif href.startswith('/blog/'):
                href = base_url + href
            print("🔗 Found href:", href)

            if base_url in href and '/blog/' in href:
                if href not in links:
                    links.add(href)
                    print("✅ Accepted:", href)
                else:
                    print("⚠️ Duplicate:", href)
            else:
                print("❌ Rejected:", href)

        page += 1

    driver.quit()
    return sorted(list(links))


In [17]:
blog_links = get_blog_links()
print(f"✅ Found {len(blog_links)} blog posts.")


NameError: name 'get_blog_links' is not defined

In [18]:
import pandas as pd

def save_blog_links(blog_links, filename="blog_links.csv"):
    df_links = pd.DataFrame(blog_links, columns=["url"])
    df_links.to_csv(filename, index=False)
    print(f"✅ Saved {len(df_links)} blog links to {filename}")


def load_blog_links(filename="blog_links.csv"):
    df_links = pd.read_csv(filename)
    return df_links["url"].tolist()

In [51]:
save_blog_links(blog_links)
# saves to a csv in wd so we don't have to wait again 

✅ Saved 395 blog links to blog_links.csv


In [19]:
# in future we can just load up the pre-scraped list 
blog_links = load_blog_links()
print(f"✅ Loaded {len(blog_links)} blog links from file")


✅ Loaded 395 blog links from file


In [63]:
def extract_info_from_post(url, driver):
    from bs4 import BeautifulSoup
    import re
    import time

    sightings = []

    # Load page
    driver.get(url)
    time.sleep(2)

    # Remove popup
    try:
        driver.execute_script("""
            var popup = document.getElementById('PopupSignupForm_0');
            if (popup) popup.remove();
        """)
    except:
        pass

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    content_div = soup.find("div", class_="blog-body")

    if not content_div:
        print(f"⚠️ Could not find blog body on {url}")
        return []

    text = content_div.get_text(separator=' ')
    text = re.sub(r'\s+', ' ', text)
    title = soup.title.string.strip() if soup.title else ""

    trip_start = extract_trip_start_date(text)
    day_sections = split_by_day_sections(text)
    print(f"🔍 Found {len(day_sections)} day sections")
    if not day_sections:
        print("⚠️ No day sections found. Falling back to whole post body.")
        day_sections = [(1, "Full Post", text)]

    for day_number, day_header, day_text in day_sections:
        day_text_clean = re.sub(r'\s+', ' ', day_text).strip()
        sighting_date = trip_start + timedelta(days=day_number - 1) if trip_start else None

        matched_site = (
            match_dive_site_fuzzy(day_header, dive_sites) or
            match_dive_site_fuzzy(day_text_clean, dive_sites)
        )

        content_lower = day_text_clean.lower()

        for kw in keywords:
            if kw in content_lower:
                for match in re.finditer(rf"\b{re.escape(kw)}\w*\b", content_lower):
                    start_idx, end_idx = match.span()
                    window_start = max(0, start_idx - 100)
                    window_end = min(len(day_text_clean), end_idx + 100)
                    excerpt = day_text_clean[window_start:window_end].strip()

                    # print(f"✅ Found '{kw}' in text at index {match.start()} — adding to sightings")

                    sightings.append({
                        "species": kw,
                        "sighting_date": sighting_date,
                        "dive_site": matched_site,
                        "url": url,
                        "title": title,
                        "excerpt": excerpt
                    })

    return sightings


In [64]:
all_sightings = []
for i, link in enumerate(blog_links):
    print(f"Scraping {i+1}/{len(blog_links)}: {link}")
    try:
        sightings = extract_info_from_post(link, driver)
        all_sightings.extend(sightings)
    except Exception as e:
        print(f"⚠️ Error on {link}: {e}")



Scraping 1/395: https://www.thesmilingseahorse.com/blog/-blotched-stingray-vs-nurse-shark
🔍 Found 0 day sections
⚠️ No day sections found. Falling back to whole post body.
✅ Found 'manta' in text at index 3743 — adding to sightings
✅ Found 'manta' in text at index 3761 — adding to sightings
✅ Found 'manta' in text at index 3781 — adding to sightings
✅ Found 'manta' in text at index 3856 — adding to sightings
✅ Found 'manta' in text at index 3959 — adding to sightings
✅ Found 'manta' in text at index 4243 — adding to sightings
✅ Found 'manta' in text at index 4780 — adding to sightings
✅ Found 'manta' in text at index 5604 — adding to sightings
✅ Found 'manta' in text at index 5693 — adding to sightings
✅ Found 'manta' in text at index 5765 — adding to sightings
✅ Found 'manta' in text at index 5951 — adding to sightings
✅ Found 'manta' in text at index 6116 — adding to sightings
✅ Found 'eagle ray' in text at index 2125 — adding to sightings
✅ Found 'stingray' in text at index 115 — ad

In [66]:
df = pd.DataFrame(all_sightings)
df.to_csv("elasmo_sightings_20250415.csv", index=False)
df.head()

,species,sighting_date,dive_site,url,title,excerpt
0,manta,None,Richelieu Rock,https://www.thesmilingseahorse.com/blog/-blotc...,"Explore Myanmar and Thailand, December 2012 - ...",ding the Christmas tree! What a great atmosphe...
1,manta,None,Richelieu Rock,https://www.thesmilingseahorse.com/blog/-blotc...,"Explore Myanmar and Thailand, December 2012 - ...","tree! What a great atmosphere, good diving, go..."
2,manta,None,Richelieu Rock,https://www.thesmilingseahorse.com/blog/-blotc...,"Explore Myanmar and Thailand, December 2012 - ...","atmosphere, good diving, good food, good peopl..."
3,manta,None,Richelieu Rock,https://www.thesmilingseahorse.com/blog/-blotc...,"Explore Myanmar and Thailand, December 2012 - ...",ck ! Manta ray encounter Manta in deep blue se...
4,manta,None,Richelieu Rock,https://www.thesmilingseahorse.com/blog/-blotc...,"Explore Myanmar and Thailand, December 2012 - ...",tas for this upcoming dive! We go to Koh Tacha...


In [23]:
driver.get("https://www.thesmilingseahorse.com/blog/trip-report-exploring-burma-banks-with-mv-smiling-seahorse-23rd-29th-january-2025")
print(driver.title)
print(driver.page_source[:2000])


Manta Rays & Shark Encounters: Diving Burma Banks with MV Smiling Seahorse (Trip Report: Jan 23-29, 2025) - Diving liveaboard in Thailand and Myanmar
<html lang="en" xmlns:fb="http://ogp.me/ns/fb#"><head>
		<title>Manta Rays &amp; Shark Encounters: Diving Burma Banks with MV Smiling Seahorse (Trip Report: Jan 23-29, 2025) - Diving liveaboard in Thailand and Myanmar</title><meta property="og:site_name" content="Diving liveaboard in Thailand and Myanmar">
<meta property="og:title" content="Manta Rays &amp; Shark Encounters: Diving Burma Banks with MV Smiling Seahorse (Trip Report: Jan 23-29, 2025)">
<meta property="og:description" content="Discover the highlights of our latest Burma Banks expedition aboard MV Smiling Seahorse! From close encounters with a manta ray and many shark species, to mesmerizing blackwater dives and the removal of a ghost net, this trip report (Jan 23-29, 2025) capt">
<meta property="og:image" content="http://www.thesmilingseahorse.com/uploads/1/0/1/8/10183628/un

In [28]:
url = "https://www.thesmilingseahorse.com/blog/trip-report-exploring-burma-banks-with-mv-smiling-seahorse-23rd-29th-january-2025"
driver.get(url)

from bs4 import BeautifulSoup
import re

soup = BeautifulSoup(driver.page_source, "html.parser")
content_div = soup.find("div", class_="blog-body")

if content_div:
    blog_text = content_div.get_text(separator=" ")
    print(blog_text[:1500])  # check if it contains "Day 1", "nurse shark", etc.
else:
    print("⚠️ Could not find blog post content.")




   
 Blog about diving in Burma and Thailand
 
 
 
 Trip report : Exploring Burma Banks with MV Smiling Seahorse ( 23rd-29th January 2025). 
 
 
 
		20/2/2025
	 
 
 
 
		0 Comments
	 
 
 
   
 
             ​ Departing from Ranong Bodin Pier, our journey to the Burma Banks began with the usual immigration formalities in Thailand before heading to Kawthung, Burma. Once the official paperwork was complete, we set course for the remote and breathtaking Mergui Archipelago and the Burma Banks. With guests from Thailand, Burma, France, Denmark, Switzerland, the USA, Italy, Panama, and Belgium, we embarked on another unforgettable adventure aboard the MV Smiling Seahorse, eager to uncover the wonders of this legendary dive destination. The  Burma Banks  is an isolated offshore reef system consisting of vast submerged plateaus and seamounts that rise abruptly from the deep ocean floor. Unlike the nearshore reefs of the Mergui Archipelago, which are fringed with islands, the Burma Banks lie fa

In [60]:
test_url = "https://www.thesmilingseahorse.com/blog/trip-report-exploring-burma-banks-with-mv-smiling-seahorse-23rd-29th-january-2025"
test_sightings = extract_info_from_post(test_url, driver)
test_df = pd.DataFrame(test_sightings)
test_df[["species", "sighting_date", "dive_site", "excerpt"]]


🔍 Found 0 day sections
⚠️ No day sections found. Falling back to whole post body.
✅ Found 'manta' in text at index 1690 — adding to sightings
✅ Found 'manta' in text at index 5425 — adding to sightings
✅ Found 'manta' in text at index 5517 — adding to sightings
✅ Found 'eagle ray' in text at index 3399 — adding to sightings
✅ Found 'stingray' in text at index 1540 — adding to sightings
✅ Found 'stingray' in text at index 1629 — adding to sightings
✅ Found 'stingray' in text at index 4594 — adding to sightings
✅ Found 'ray' in text at index 1696 — adding to sightings
✅ Found 'ray' in text at index 2463 — adding to sightings
✅ Found 'ray' in text at index 3405 — adding to sightings
✅ Found 'ray' in text at index 4755 — adding to sightings
✅ Found 'ray' in text at index 5309 — adding to sightings
✅ Found 'ray' in text at index 5431 — adding to sightings
✅ Found 'ray' in text at index 8014 — adding to sightings
✅ Found 'ray' in text at index 8325 — adding to sightings
✅ Found 'shark' in te

,species,sighting_date,dive_site,excerpt
0,manta,2025-01-23,North Twin,dy seabed hosts garden eels and hidden stingra...
1,manta,2025-01-23,North Twin,orgettable sighting of three massive requiem s...
2,manta,2025-01-23,North Twin,"oceanic manta ray passing by to say hello, a r..."
3,eagle ray,2025-01-23,North Twin,"octopuses changing colors, a powerful school o..."
4,stingray,2025-01-23,North Twin,views of the underwater landscape. Divers can ...
5,stingray,2025-01-23,North Twin,"ef sharks, stingrays, and schools of jackfish ..."
6,stingray,2025-01-23,North Twin,and a coral crab protecting its orange eggs. T...
7,ray,2025-01-23,North Twin,bed hosts garden eels and hidden stingrays . O...
8,ray,2025-01-23,North Twin,s. ​ Our third dive at Shark Cave delivered mo...
9,ray,2025-01-23,North Twin,"uses changing colors, a powerful school of gia..."


In [61]:
test_url = "https://www.thesmilingseahorse.com/blog/trip-report-exploring-burma-banks-with-mv-smiling-seahorse-23rd-29th-january-2025"
test_sightings = extract_info_from_post(test_url, driver)

print("✅ Sightings:", len(test_sightings))
if test_sightings:
    for s in test_sightings:
        print(s)
else:
    print("⚠️ No matches found.")



🔍 Found 0 day sections
⚠️ No day sections found. Falling back to whole post body.
✅ Found 'manta' in text at index 1690 — adding to sightings
✅ Found 'manta' in text at index 5425 — adding to sightings
✅ Found 'manta' in text at index 5517 — adding to sightings
✅ Found 'eagle ray' in text at index 3399 — adding to sightings
✅ Found 'stingray' in text at index 1540 — adding to sightings
✅ Found 'stingray' in text at index 1629 — adding to sightings
✅ Found 'stingray' in text at index 4594 — adding to sightings
✅ Found 'ray' in text at index 1696 — adding to sightings
✅ Found 'ray' in text at index 2463 — adding to sightings
✅ Found 'ray' in text at index 3405 — adding to sightings
✅ Found 'ray' in text at index 4755 — adding to sightings
✅ Found 'ray' in text at index 5309 — adding to sightings
✅ Found 'ray' in text at index 5431 — adding to sightings
✅ Found 'ray' in text at index 8014 — adding to sightings
✅ Found 'ray' in text at index 8325 — adding to sightings
✅ Found 'shark' in te

In [62]:
from bs4 import BeautifulSoup
import re

driver.get(test_url)
soup = BeautifulSoup(driver.page_source, "html.parser")
div = soup.find("div", class_="blog-body")

if div:
    text = div.get_text(separator=" ")
    text = re.sub(r"\s+", " ", text)
    matches = [kw for kw in keywords if kw in text.lower()]
    print("✅ Keywords matched:", matches)
else:
    print("⚠️ No blog content found.")



✅ Keywords matched: ['manta', 'eagle ray', 'stingray', 'ray', 'shark', 'reef shark', 'blacktip', 'nurse shark', 'sharks', 'rays', 'silvertip', 'requiem', 'oceanic manta', 'sharks']
